In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import torch
import matplotlib.pyplot as plt
# from dask.distributed import Client

import climex_utils as cu
import train_prob_unet_model as tm  
from prob_unet import ProbabilisticUNet
from prob_unet_utils import plot_losses, plot_losses_mae
import pickle
import numpy as np
import random
import os

In [ ]:

if __name__ == "__main__":

    def set_seed(seed):
        random.seed(seed) 
        np.random.seed(seed)  
        torch.manual_seed(seed) 
        torch.cuda.manual_seed(seed)  
        torch.cuda.manual_seed_all(seed)  
        torch.backends.cudnn.deterministic = True  
        torch.backends.cudnn.benchmark = False  
        os.environ['PYTHONHASHSEED'] = str(seed)

    # Set seed for reproducibility   
    set_seed(42)  

    # Importing all required arguments
    args = tm.get_args()
    args.lowres_scale = 16
    args.num_epochs = 30
    args.batch_size = 64

    # Initializing the Probabilistic UNet model
    probunet_model = ProbabilisticUNet(
        input_channels=len(args.variables),
        num_classes=len(args.variables),
        latent_dim=16,
        num_filters=[32, 64, 128, 256],
        model_channels=32,
        channel_mult= [1, 2, 4, 8],
        beta_0=0.0,
        beta_1=0.0,
        beta_2=0.0  
    ).to(args.device)

    # Initializing the datasets
    dataset_train = cu.climex2torch(
        datadir=args.datadir,
        years=args.years_train,
        variables=args.variables,
        type="lrinterp_to_residuals",
        transfo=True,
        coords=args.coords,
        lowres_scale=args.lowres_scale
    )
    
    dataset_val = cu.climex2torch(
        datadir=args.datadir,
        years=args.years_val,
        variables=args.variables,
        coords=args.coords,
        lowres_scale=args.lowres_scale,
        type="lrinterp_to_residuals",
        transfo=True
    )
    dataset_test = cu.climex2torch(
        datadir=args.datadir,
        years=args.years_test,
        variables=args.variables,
        coords=args.coords,
        lowres_scale=args.lowres_scale,
        type="lrinterp_to_residuals",
        transfo=True
    )

    # Initializing the dataloaders
    dataloader_train = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=0
    )
    dataloader_val = torch.utils.data.DataLoader(
        dataset_val,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=0
    )
    dataloader_test = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=0
    )
    dataloader_test_random = torch.utils.data.DataLoader(
        dataset_val,
        batch_size=2,
        shuffle=True,
        num_workers=0
    )

    # Initializing training objects
    optimizer = args.optimizer(params=probunet_model.parameters(), lr=args.lr)
    # optimizer = torch.optim.Adam(probunet_model.parameters(), lr=args.lr, weight_decay=1e-4)


    # Initialize loss tracking lists for each variable
    tr_losses_mae = {var: [] for var in args.variables}
    tr_losses_kl = {var: [] for var in args.variables}
    tr_losses_kl2 = {var: [] for var in args.variables}
    val_losses_mae = {var: [] for var in args.variables}
    val_losses_kl = {var: [] for var in args.variables}
    val_losses_kl2 = {var: [] for var in args.variables}

    beta_0 = 1.0
    beta_1 = 0.00
    beta_2 = 0.00
        
    warmup_epochs = 1
    # Training loop
    print(f"Probabilistic Unet Latent dim: {probunet_model.latent_dim}")
    for epoch in range(1, args.num_epochs + 1):

        probunet_model.beta_0 = beta_0
        probunet_model.beta_1 = beta_1
        probunet_model.beta_2 = beta_2

    
        print(f"Epoch {epoch}/{args.num_epochs} - beta_0: {probunet_model.beta_0}, beta_1: {probunet_model.beta_1:.4f}, beta_2: {probunet_model.beta_2:.4f}")

        # Training for one epoch
        train_losses_mae, training_losses_kl, training_losses_kl2, kl_div, kl_div2 = tm.train_probunet_step(
            model=probunet_model,
            dataloader=dataloader_train,
            optimizer=optimizer,
            epoch=epoch,
            num_epochs=args.num_epochs,
            device=args.device,
            variables=args.variables,
        )
        for var in args.variables:
            tr_losses_mae[var].append(train_losses_mae[var])
            tr_losses_kl[var].append(training_losses_kl[var])
            tr_losses_kl2[var].append(training_losses_kl2[var])
        
        # Compute average losses for each term
        avg_recon_loss = sum(train_losses_mae.values()) / len(train_losses_mae)  # Average reconstruction loss
        avg_kl_loss = sum(training_losses_kl.values()) / len(training_losses_kl)      # Average KL (posterior vs prior)
        avg_kl2_loss = sum(training_losses_kl2.values()) / len(training_losses_kl2)  # Average KL (posterior vs Gaussian)

        # Ensure losses are scalars by detaching and converting them
        # avg_recon_loss = float(avg_recon_loss.detach().cpu().item())  # Detach and convert to scalar
        avg_kl_loss = float(avg_kl_loss.detach().cpu().item())
        avg_kl2_loss = float(avg_kl2_loss.detach().cpu().item())
        

        if epoch > warmup_epochs:
            # beta_0 = 1.0 / (avg_recon_loss + 1e-7)  
            beta_0 = 1.0 
            beta_1 = 1.0 / (avg_kl_loss + 1e-7)
            beta_2 = 1.0 / (avg_kl2_loss + 1e-7)

        else:
            beta_0 = 1.0
            beta_1 = 0.00
            beta_2 = 0.00
        
        
        # Evaluating the model on validation data
        val_losses_mae_running, val_losses_kl_running, val_losses_kl2_running = tm.eval_probunet_model(
            model=probunet_model,
            dataloader=dataloader_val,
            reconstruct=False,
            device=args.device,           
        )
        for var in args.variables:
            val_losses_mae[var].append(val_losses_mae_running[var])
            val_losses_kl[var].append(val_losses_kl_running[var])
            val_losses_kl2[var].append(val_losses_kl2_running[var])
    
        
        test_batch = next(iter(dataloader_test_random))

        residual_preds, (fig, axs) = tm.sample_residual_probunet_model(
            model=probunet_model,
            dataloader=dataloader_test_random,
            epoch=epoch,
            device=args.device,
            batch=test_batch
        )
        fig.savefig(f"{args.plotdir}/epoch{epoch}_residuals.png", dpi=300)
        plt.close(fig)

        fig_difs, axs_difs = dataset_test.plot_residual_differences(
        residual_preds=residual_preds,
        timestamps_float=test_batch['timestamps_float'][:2],
        epoch=epoch,
        N=2, 
        num_samples=3
        )
        fig_difs.savefig(f"{args.plotdir}/epoch{epoch}_res_difs.png", dpi=300)
        plt.close(fig_difs)

        samples, (fig, axs) = tm.sample_probunet_model(
            model=probunet_model,
            dataloader=dataloader_test_random,
            epoch=epoch,
            device=args.device,
            batch=test_batch
        )
        fig.savefig(f"{args.plotdir}/epoch{epoch}_reconstructed.png", dpi=300)
        plt.close(fig)
    
    # Save losses to a file after training
    losses_to_save = {
        "train_losses_mae": tr_losses_mae,
        "train_losses_kl": tr_losses_kl,
        "train_losses_kl2": tr_losses_kl2,
        "val_losses_mae": val_losses_mae,
        "val_losses_kl": val_losses_kl,
        "val_losses_kl2": val_losses_kl2
    }
    with open(f"{args.plotdir}/losses.pkl", "wb") as f:
        pickle.dump(losses_to_save, f)

    torch.save(probunet_model.state_dict(), f"{args.plotdir}/probunet_model_lat_dim_{probunet_model.latent_dim}.pth")

    # # Plot training and validation loss curves for each variable
    # plot_losses(tr_losses_mae, tr_losses_kl, val_losses_mae, val_losses_kl, args.variables, args.plotdir)

    plot_losses(tr_losses_mae, tr_losses_kl, tr_losses_kl2, val_losses_mae, val_losses_kl, val_losses_kl2, args.variables, args.plotdir)

Opening and lazy loading netCDF files
Loading dataset into memory
Converting xarray Dataset to Pytorch tensor

##########################################
############ PROCESSING DONE #############
##########################################

Opening and lazy loading netCDF files
Loading dataset into memory
Converting xarray Dataset to Pytorch tensor

##########################################
############ PROCESSING DONE #############
##########################################

Opening and lazy loading netCDF files
Loading dataset into memory
Converting xarray Dataset to Pytorch tensor

##########################################
############ PROCESSING DONE #############
##########################################

Probabilistic Unet Latent dim: 16
Epoch 1/30 - beta_0: 1.0, beta_1: 0.0000, beta_2: 0.0000


Train :: Epoch: 1/30:   0%|                                                                                                | 0/172 [00:00<?, ?it/s]

Computing statistics for standardization


:: Evaluation :::   4%|████                                                                                         | 2/46 [00:00<00:04, 10.35it/s]

Computing statistics for standardization


...

Epoch 29/30 - beta_0: 1.0, beta_1: 9856900.6449, beta_2: 9982682.3217


:: Evaluation ::: 100%|██████████████████████████████████████████████████████████████████████████████| 46/46 [00:05<00:00,  8.35it/s, Loss: 0.1882]


Epoch 30/30 - beta_0: 1.0, beta_1: 9835760.7452, beta_2: 9980321.0321


:: Evaluation ::: 100%|██████████████████████████████████████████████████████████████████████████████| 46/46 [00:05<00:00,  8.37it/s, Loss: 0.1875]


: 